<a href="https://colab.research.google.com/github/1NetoDev/curriculAI/blob/main/CurriculAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Célula 1: Instalações e Configurações Iniciais

# Instalações de bibliotecas Python
!pip install -q gradio PyPDF2 pytesseract Pillow

# Instalação do Tesseract OCR e pacotes de linguagem (exemplo: português)
!sudo apt update
!sudo apt install -y tesseract-ocr
!sudo apt install -y tesseract-ocr-por # Para o idioma português

print("Instalações concluídas.")

# Configuração da API Key do Google AI
import os
from google.colab import userdata
import google.generativeai as genai

try:
    GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
    if GOOGLE_API_KEY is None:
        raise ValueError("Chave 'GOOGLE_API_KEY' não encontrada nos Secrets do Colab.")
    genai.configure(api_key=GOOGLE_API_KEY)
    print("Chave da API do Google AI configurada com sucesso!")
except Exception as e:
    print(f"ATENÇÃO: Erro ao configurar a chave da API do Google AI: {e}")
    print("A funcionalidade de IA não funcionará sem a chave da API.")
    # Definir GOOGLE_API_KEY como None pode ser útil para checagens posteriores,
    # mas genai.configure() já terá falhado ou não sido chamado.

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:4 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:10 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Fetched 384 kB in 1s (262 kB/s)
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
35 packages can be upgraded. Run 'apt list --upgradable' to see them.
W: Skipping acquire of configured file 'main/source/Sources' as

In [ ]:
# --- Configuração e Inicialização do Git no Google Colab ---

# 1. Navegue para o diretório onde seus arquivos do projeto estão.
#    Em muitos casos no Colab, seus arquivos estarão na raiz do /content/
#    Se você montou seu Google Drive e seus arquivos estão lá, o caminho seria diferente (ex: %cd /content/drive/MyDrive/SuaPastaDoProjeto)
%cd /content/

# 2. Configure seu nome de usuário e e-mail para os commits.
#    Substitua "Seu Nome de Usuário do GitHub" e "Seu Email do GitHub" pelos seus dados.
!git config --global user.name "Seu Nome de Usuário do GitHub"
!git config --global user.email "Seu Email do GitHub"

# 3. Inicialize um novo repositório Git no diretório atual.
#    Isso cria uma subpasta oculta chamada .git/ que rastreará suas mudanças.
!git init

# 4. Adicione todos os arquivos do diretório atual e subdiretórios à "staging area".
#    Isso diz ao Git que você quer incluir essas mudanças no próximo commit.
!git add .

# 5. Crie o primeiro commit com uma mensagem descritiva.
#    Este é o primeiro "ponto de salvamento" oficial do seu projeto no Git.
!git commit -m "Commit inicial do projeto Auxiliar de Curriculo"

# 6. Renomeie o branch principal local de 'master' (nome antigo padrão) para 'main'.
#    'main' é o nome padrão mais moderno para branches principais.
!git branch -M main

# 7. Adicione o link para o seu repositório VAZIO no GitHub como um "remote".
#    'origin' é apenas um apelido para essa URL remota.
#    SUBSTITUA a URL de exemplo pela URL HTTPS do seu repositório NO GITHUB.
github_repo_url = "https://github.com/seu-usuario/nome-do-repositorio.git" # <-- SUBSTITUA AQUI!
!git remote add origin {github_repo_url}
# Se você já tentou adicionar o remote antes e deu erro, use a linha abaixo em vez da anterior:
# !git remote set-url origin {github_repo_url}


# 8. Envie seus commits locais (do branch 'main') para o repositório remoto no GitHub ('origin').
#    O '-u' define que seu branch local 'main' deve "rastrear" o branch 'main' no remoto.
#    VOCÊ SERÁ SOLICITADO AQUI! Use seu NOME DE USUÁRIO do GitHub e o TOKEN DE ACESSO PESSOAL como SENHA.
!git push -u origin main

# --- Fim da Configuração ---

filter.lfs.required=true
filter.lfs.clean=git-lfs clean -- %f
filter.lfs.smudge=git-lfs smudge -- %f
filter.lfs.process=git-lfs filter-process
user.name=1NetoDev
user.email=anderaes@gmail.com


In [ ]:
# Célula 2: Funções de Backend (Lógica Principal) - REVISADA

import PyPDF2
from PIL import Image
import pytesseract
import google.generativeai as genai

# --- Função para enviar prompt à IA ---
def enviar_prompt_para_ia(prompt_completo, instrucao_sistema_str, nome_modelo_str="gemini-1.5-flash-latest"):
    print(f"DEBUG - enviar_prompt_para_ia: Tentando enviar prompt para o modelo '{nome_modelo_str}'.")
    try:
        model = genai.GenerativeModel(
            model_name=nome_modelo_str,
            system_instruction=instrucao_sistema_str
        )
        response = model.generate_content(prompt_completo)
        # Para depurar a resposta completa da API, se necessário:
        # print(f"DEBUG - enviar_prompt_para_ia: Resposta da API: {response}")
        if not response.parts:
             print("DEBUG - enviar_prompt_para_ia: Resposta da IA não contém 'parts'. Possível erro ou resposta vazia.")
             return "ErroAPI_RespostaInesperadaOuVazia"
        print(f"DEBUG - enviar_prompt_para_ia: Resposta da IA recebida.")
        return response.text # .text acessa o conteúdo textual da primeira parte
    except AttributeError as ae:
        if "'NoneType' object has no attribute 'send_request'" in str(ae) or \
           "configure() has not been called" in str(ae) or \
           "'GoogleGenerativeAI' object has no attribute 'default_transport'" in str(ae):
            print("DEBUG - enviar_prompt_para_ia: Erro crítico - API Key provavelmente não configurada via genai.configure().")
            return "ErroAPI_ChaveNaoConfiguradaOuInvalida"
        else:
            print(f"DEBUG - enviar_prompt_para_ia: Erro de Atributo ao chamar a API Gemini: {ae}")
            return f"ErroAPI_Atributo: {str(ae)}"
    except Exception as e:
        print(f"DEBUG - enviar_prompt_para_ia: Erro ao chamar a API Gemini: {e}")
        # Tentar obter mais detalhes do erro da resposta, se disponível
        error_details = getattr(e, 'message', str(e))
        if hasattr(e, 'response') and hasattr(e.response, 'text'):
            error_details += f" | Detalhes da API: {e.response.text}"
        return f"ErroAPI_Comunicacao: {error_details}"

# --- Funções de Leitura de Arquivo (ler_pdf, ler_imagem, obter_texto_entrada) ---
# (MANTENHA AS VERSÕES ANTERIORES DESTAS FUNÇÕES AQUI, elas já estão boas
# com o tratamento de ErroInterno_)

def ler_pdf(caminho_pdf):
    # ... (código da sua função ler_pdf) ...
    print(f"DEBUG - ler_pdf: Tentando ler '{caminho_pdf}'")
    try:
        texto_completo = ""
        with open(caminho_pdf, 'rb') as arquivo_pdf:
            leitor_pdf = PyPDF2.PdfReader(arquivo_pdf)
            if not leitor_pdf.pages:
                 print(f"DEBUG - ler_pdf: PDF '{caminho_pdf}' não tem páginas ou está vazio.")
                 return "ErroInterno_PDFVazioOuSemPaginas"
            for pagina_num in range(len(leitor_pdf.pages)):
                pagina = leitor_pdf.pages[pagina_num]
                texto_pagina = pagina.extract_text()
                if texto_pagina:
                    texto_completo += texto_pagina + "\n"
        if not texto_completo.strip():
            print(f"DEBUG - ler_pdf: Nenhum texto extraído do PDF '{caminho_pdf}'. Pode ser um PDF de imagem sem OCR.")
            return "ErroInterno_PDFSemTextoUtil"
        texto_preview = texto_completo[:50].replace('\n', ' ')
        print(f"DEBUG - ler_pdf: Texto extraído de '{caminho_pdf}' (primeiros 50 chars): '{texto_preview}...'")
        return texto_completo
    except FileNotFoundError:
        print(f"DEBUG - ler_pdf: Arquivo não encontrado em '{caminho_pdf}'.")
        return "ErroInterno_ArquivoNaoEncontrado"
    except PyPDF2.errors.PdfReadError as e:
        print(f"DEBUG - ler_pdf: Erro ao ler o PDF '{caminho_pdf}': {e}")
        return f"ErroInterno_LeituraPDF: {str(e)}"
    except Exception as e:
        print(f"DEBUG - ler_pdf: Erro inesperado ao ler o PDF '{caminho_pdf}': {e}")
        return f"ErroInterno_InesperadoPDF: {str(e)}"

def ler_imagem(caminho_imagem):
    # ... (código da sua função ler_imagem) ...
    print(f"DEBUG - ler_imagem: Tentando ler '{caminho_imagem}' com Tesseract.")
    try:
        texto_extraido = pytesseract.image_to_string(Image.open(caminho_imagem), lang='por')
        if not texto_extraido.strip():
            print(f"DEBUG - ler_imagem: Nenhum texto extraído da imagem '{caminho_imagem}'.")
            return "ErroInterno_ImagemSemTextoUtil"
        texto_preview = texto_extraido[:50].replace('\n', ' ')
        print(f"DEBUG - ler_imagem: Texto extraído de '{caminho_imagem}' (primeiros 50 chars): '{texto_preview}...'")
        return texto_extraido
    except FileNotFoundError:
        print(f"DEBUG - ler_imagem: Arquivo não encontrado em '{caminho_imagem}'.")
        return "ErroInterno_ArquivoNaoEncontrado"
    except pytesseract.TesseractNotFoundError:
        print("DEBUG - ler_imagem: Erro Crítico - Tesseract OCR não encontrado. Verifique a instalação na Célula 1.")
        return "ErroInterno_TesseractNaoEncontrado"
    except Exception as e:
        print(f"DEBUG - ler_imagem: Erro inesperado ao ler a imagem '{caminho_imagem}': {e}")
        return f"ErroInterno_InesperadoImagem: {str(e)}"

def obter_texto_entrada(entrada_raw, tipo_entrada_str):
    # ... (código da sua função obter_texto_entrada) ...
    print(f"DEBUG - obter_texto_entrada: recebendo entrada_raw (tipo: {type(entrada_raw)}), tipo_entrada_str: '{tipo_entrada_str}'")
    if tipo_entrada_str == "texto":
        if not isinstance(entrada_raw, str):
             print(f"DEBUG - obter_texto_entrada: Esperava string para tipo 'texto', mas recebeu {type(entrada_raw)}")
             return "ErroInterno_TipoIncorretoParaTexto"
        return entrada_raw
    elif tipo_entrada_str == "pdf":
        return ler_pdf(entrada_raw) # entrada_raw aqui é o caminho do arquivo
    elif tipo_entrada_str == "imagem":
        return ler_imagem(entrada_raw) # entrada_raw aqui é o caminho do arquivo
    else:
        msg_erro = f"ErroInterno_TipoEntradaDesconhecido: '{tipo_entrada_str}'"
        print(f"DEBUG - obter_texto_entrada: {msg_erro}")
        return msg_erro

# --- Função Principal de Análise ---
def analisar_curriculo_completo_para_vaga(curriculo_input_raw, vaga_input_raw, tipo_curriculo_str, tipo_vaga_str):
    nome_funcao_analise = "analisar_curriculo_completo_para_vaga"
    print(f"DEBUG - {nome_funcao_analise}: Iniciando com currículo (tipo: {tipo_curriculo_str}) e vaga (tipo: {tipo_vaga_str}).")

    texto_do_curriculo = obter_texto_entrada(curriculo_input_raw, tipo_curriculo_str)
    if isinstance(texto_do_curriculo, str) and texto_do_curriculo.startswith("ErroInterno_"):
        print(f"DEBUG - {nome_funcao_analise}: Erro ao obter texto do currículo: {texto_do_curriculo}")
        return f"Não foi possível processar o arquivo do currículo. (Detalhe: {texto_do_curriculo})"

    texto_da_vaga = ""
    if vaga_input_raw and (isinstance(vaga_input_raw, str) and vaga_input_raw.strip()): # Checa se é string e não vazia
        texto_da_vaga = obter_texto_entrada(vaga_input_raw, tipo_vaga_str)
        if isinstance(texto_da_vaga, str) and texto_da_vaga.startswith("ErroInterno_"):
            print(f"DEBUG - {nome_funcao_analise}: Erro ao obter texto da vaga: {texto_da_vaga}")
            return f"Não foi possível processar o arquivo da descrição da vaga. (Detalhe: {texto_da_vaga})"
    else:
        print(f"DEBUG - {nome_funcao_analise}: Nenhuma descrição de vaga fornecida ou entrada vazia para vaga.")
        texto_da_vaga = "" # Garante que é uma string vazia se não processada

    # NOVO PROMPT PRINCIPAL (System Instruction)
    instrucao_de_sistema = """
Você é um(a) **Especialista Sênior em Recrutamento e Desenvolvimento de Carreira**, com mais de 15 anos de experiência em grandes empresas e consultorias de RH. Possui um conhecimento profundo sobre o que os recrutadores e sistemas ATS (Applicant Tracking Systems) procuram em um currículo. Além da experiência prática, você domina **princípios de psicologia aplicada a processos seletivos e comunicação profissional**, sabendo exatamente como um currículo pode causar a melhor primeira impressão, manter a atenção e persuadir o leitor (seja ele humano ou uma máquina).

Seu objetivo principal é **ajudar o usuário a otimizar o currículo dele para uma vaga específica**, aumentando significativamente as chances de ele ser notado e chamado para uma entrevista. Você não apenas aponta o que deve ser mudado, mas **explica o porquê** da mudança, usando sua expertise em RH e psicologia.

Você receberá o **texto do currículo atual do usuário** e, opcionalmente, o **texto da descrição da vaga** para a qual ele deseja se candidatar.

Sua análise e sugestões devem ser:

1.  **Altamente Relevantes:** Se uma vaga for fornecida, foque *exclusivamente* em adequar o currículo à *vaga específica*. Ignore informações no currículo que não são relevantes para esta vaga e destaque as que são. Se nenhuma vaga for fornecida, ofereça melhorias gerais para tornar o currículo mais forte e versátil.
2.  **Acionáveis e Específicas:** Não dê conselhos genéricos. Mostre *exatamente* onde e como o currículo pode ser melhorado (sugestões de reescrita, inclusão, remoção).
3.  **Baseadas em Expertise de RH e Psicologia:**
    * **ATS:** Se uma vaga for fornecida, identifique palavras-chave e termos essenciais da vaga e sugira onde inseri-los naturalmente no currículo.
    * **Recrutador Humano (Psicologia):**
        * Sugira um **Título/Resumo Profissional** que capture a atenção nos primeiros segundos, sendo direto e relevante (para a vaga, se houver, ou de forma geral).
        * Analise as descrições de experiência: Ajude o usuário a transformar responsabilidades em **conquistas e resultados quantificáveis**, usando verbos de ação fortes. (Ex: Em vez de "Gerenciei o projeto X", sugira "Liderei o projeto X, reduzindo custos em Y% e entregando Z antes do prazo." - Explique que recrutadores buscam impacto).
        * Analise o **tom e o vocabulário:** Sugira ajustes para que a linguagem seja profissional, confiante e alinhada com a cultura da área/empresa (se a vaga der pistas, ou de forma geral). Explique o impacto psicológico de um tom adequado.
        * Sugira a melhor forma de apresentar habilidades, focando nas mais relevantes (para a vaga ou gerais) e naquelas que causam maior impacto.
        * Analise a **clareza e a escaneabilidade:** Sugira como usar formatação (quebras de linha, tópicos - você pode instruir sobre isso) e linguagem concisa.
    * **Estrutura:** Comente textualmente sobre a estrutura ideal do currículo (para a vaga ou de forma geral), se a estrutura atual dificultar a visualização das informações chave.
4.  **Explicativas:** Sempre justifique suas sugestões conectando-as aos critérios de RH (ATS, clareza, impacto, relevância) e, quando aplicável, aos princípios de psicologia.

**Formato de Saída:**
Apresente suas sugestões de forma clara e bem organizada. Use **negrito** para destacar termos chave ou títulos de seção que você criar, *itálico* para ênfase ou exemplos, e listas com hífens (`-`) para itens de sugestão.
Inicie com uma breve saudação e uma visão geral do que será analisado. Finalize com uma mensagem encorajadora.

**Exemplo de Sugestão Formatada:**
- Na seção "**Experiência Profissional**" para o cargo de *Desenvolvedor Frontend na Empresa ABC*:
  - O ponto: "Responsável pelo desenvolvimento de interfaces" poderia ser aprimorado para: "**Desenvolvi e implementei interfaces de usuário responsivas para 3 projetos web, resultando em um aumento de 15% na satisfação do cliente.**"
    *   ***Porquê:*** *Esta reformulação usa um verbo de ação forte ("Desenvolvi e implementei"), quantifica o impacto ("3 projetos", "aumento de 15%") e foca em resultados, o que é altamente valorizado por recrutadores e mais eficaz para demonstrar suas capacidades.*
    """

    # Construção do Prompt do Usuário
    prompt_usuario = []
    prompt_usuario.append("Por favor, analise meu currículo e forneça sugestões de otimização.")

    prompt_usuario.append("\n\n--- MEU CURRÍCULO ATUAL ---")
    prompt_usuario.append(texto_do_curriculo if texto_do_curriculo and not texto_do_curriculo.startswith("ErroInterno_") else "Não foi possível carregar o conteúdo do currículo.")

    if texto_da_vaga and texto_da_vaga.strip() and not texto_da_vaga.startswith("ErroInterno_"):
        prompt_usuario.append("\n\n--- DESCRIÇÃO DA VAGA ALVO ---")
        prompt_usuario.append(texto_da_vaga)
    else:
        prompt_usuario.append("\n\n(Nenhuma descrição de vaga específica foi fornecida. Por favor, forneça melhorias gerais para o currículo.)")

    prompt_final_para_ia = "\n".join(prompt_usuario)

    prompt_preview = prompt_final_para_ia[:300].replace('\n', ' ') # Aumentado o preview
    print(f"DEBUG - {nome_funcao_analise}: Prompt do usuário (primeiros 300 chars): '{prompt_preview}...'")

    sugestoes_da_ia = enviar_prompt_para_ia(prompt_final_para_ia, instrucao_de_sistema)

    if isinstance(sugestoes_da_ia, str) and sugestoes_da_ia.startswith("ErroAPI_"):
        print(f"DEBUG - {nome_funcao_analise}: Erro da IA: {sugestoes_da_ia}")
        return f"Ocorreu um erro ao comunicar com a IA para análise completa: {sugestoes_da_ia}"

    sugestoes_preview = str(sugestoes_da_ia)[:200].replace('\n', ' ') # Aumentado o preview
    print(f"DEBUG - {nome_funcao_analise}: Sugestões recebidas (primeiros 200 chars): '{sugestoes_preview}...'")
    return sugestoes_da_ia

print("Funções de backend (revisadas) definidas.")

Funções de backend (revisadas) definidas.


In [ ]:
# Célula 3: Funções Wrapper para a Interface Gradio - REVISADA
import os # Para os.path.splitext

# Função de processamento principal para a interface
def processar_analise_completa_ui(tipo_input_selecionado, curriculo_file_obj, curriculo_texto_input, vaga_file_obj, vaga_texto_input):
    nome_funcao_ui = "processar_analise_completa_ui"
    print(f"--- Iniciando {nome_funcao_ui} com tipo_input: {tipo_input_selecionado} ---")

    curriculo_final_input = ""
    tipo_curriculo = ""
    descricao_vaga_final_input = ""
    tipo_vaga = ""
    erro_msg = []

    # Processar Currículo
    if tipo_input_selecionado == "Arquivo":
        if curriculo_file_obj is not None:
            curriculo_final_input = curriculo_file_obj.name
            ext = os.path.splitext(curriculo_final_input)[1].lower()
            if ext == ".pdf": tipo_curriculo = "pdf"
            elif ext in [".png", ".jpg", ".jpeg", ".webp"]: tipo_curriculo = "imagem"
            else: erro_msg.append("Erro no Currículo: Tipo de arquivo não suportado. Use PDF ou Imagem.")
            print(f"DEBUG UI: Currículo por arquivo: {curriculo_final_input}, tipo: {tipo_curriculo}")
        else:
            erro_msg.append("Erro: Nenhum arquivo de currículo foi enviado.")
    elif tipo_input_selecionado == "Texto":
        if curriculo_texto_input and curriculo_texto_input.strip():
            curriculo_final_input = curriculo_texto_input
            tipo_curriculo = "texto"
            print(f"DEBUG UI: Currículo por texto.")
        else:
            erro_msg.append("Erro: O campo de texto do currículo está vazio.")
    else: # Caso inesperado
        erro_msg.append("Erro: Seleção de tipo de entrada para currículo inválida.")


    # Processar Descrição da Vaga (opcional, e segue a mesma lógica de tipo de input do currículo)
    # Assumiremos por simplicidade que a vaga também segue o tipo_input_selecionado se fornecida como arquivo,
    # ou pode ser texto independentemente.
    # Uma UI mais granular teria seletores de tipo separados para currículo e vaga.

    if vaga_file_obj is not None: # Prioriza arquivo se ambos forem fornecidos para a vaga
        descricao_vaga_final_input = vaga_file_obj.name
        ext_vaga = os.path.splitext(descricao_vaga_final_input)[1].lower()
        if ext_vaga == ".pdf": tipo_vaga = "pdf"
        elif ext_vaga in [".png", ".jpg", ".jpeg", ".webp"]: tipo_vaga = "imagem"
        else: erro_msg.append("Erro na Vaga: Tipo de arquivo da vaga não suportado.")
        print(f"DEBUG UI: Vaga por arquivo: {descricao_vaga_final_input}, tipo: {tipo_vaga}")
    elif vaga_texto_input and vaga_texto_input.strip():
        descricao_vaga_final_input = vaga_texto_input
        tipo_vaga = "texto"
        print(f"DEBUG UI: Vaga por texto.")
    else:
        descricao_vaga_final_input = "" # Vaga opcional
        tipo_vaga = "texto" # Default se não fornecida
        print(f"DEBUG UI: Vaga não fornecida.")


    if erro_msg: # Se houve erros de validação dos inputs da UI
        print(f"DEBUG UI: Erros de validação: {erro_msg}")
        return "\n".join(erro_msg) # Retorna a mensagem de erro para a UI

    print(f"DEBUG UI: Chamando backend 'analisar_curriculo_completo_para_vaga'")
    try:
        # Chamada para a nova função de backend unificada
        resultado_analise = analisar_curriculo_completo_para_vaga(
            curriculo_final_input,
            descricao_vaga_final_input,
            tipo_curriculo,
            tipo_vaga
        )
        print(f"DEBUG UI: Resultado da análise completa recebido.")
        return resultado_analise # Este resultado já deve vir formatado com Markdown pela IA
    except Exception as e:
        import traceback
        print(f"DEBUG UI: Exceção CAPTURADA em {nome_funcao_ui} ao chamar o backend: {str(e)}")
        traceback.print_exc()
        return f"Ocorreu um erro inesperado na interface ao processar a análise: {str(e)}"

print("Função wrapper da interface (revisada) definida.")

Função wrapper da interface (revisada) definida.


In [ ]:
# Célula 4: Definição e Lançamento da Interface Gradio - REVISADA
import gradio as gr
# import os # Não é mais estritamente necessário aqui se a lógica de UI não manipular caminhos

# Definindo o tema
theme = gr.themes.Soft(
    primary_hue="orange",
    secondary_hue="yellow",
    neutral_hue="slate",
    font=[gr.themes.GoogleFont("Inter"), "ui-sans-serif", "system-ui", "sans-serif"],
)

with gr.Blocks(theme=theme, title="Auxiliar de Currículo IA") as demo:
    gr.Markdown(
        """
        <div style="text-align: center;">
            <h1 style="color: #E67E22;">🚀 Auxiliar de Currículo Inteligente 🚀</h1>
            <p>Potencialize seu currículo com Inteligência Artificial! Forneça seu currículo e, opcionalmente, a descrição da vaga desejada.</p>
        </div>
        """
    )

    with gr.Row():
        input_type_selector = gr.Radio(
            choices=["Arquivo", "Texto"],
            label="Como deseja fornecer o currículo e a vaga?",
            value="Arquivo", # Valor padrão
            interactive=True
        )

    # --- Inputs para Currículo ---
    with gr.Group(visible=True) as group_curriculo_arquivo: # Começa visível se 'Arquivo' for padrão
        curriculo_arquivo_input = gr.File(label="Upload do SEU CURRÍCULO (PDF/Imagem)", file_types=[".pdf", ".png", ".jpg", ".jpeg", ".webp"])

    with gr.Group(visible=False) as group_curriculo_texto: # Começa invisível
        curriculo_texto_input = gr.Textbox(label="Cole o texto do SEU CURRÍCULO aqui", lines=10, placeholder="Cole todo o conteúdo do seu currículo...")

    # --- Inputs para Descrição da Vaga (Opcional) ---
    gr.Markdown("---") # Separador visual
    gr.Markdown("**Descrição da Vaga (Opcional)**")

    with gr.Group(visible=True) as group_vaga_arquivo: # Começa visível se 'Arquivo' for padrão
        vaga_arquivo_input = gr.File(label="Upload da DESCRIÇÃO DA VAGA (PDF/Imagem)", file_types=[".pdf", ".png", ".jpg", ".jpeg", ".webp"])

    with gr.Group(visible=False) as group_vaga_texto: # Começa invisível
        vaga_texto_input = gr.Textbox(label="Cole o texto da DESCRIÇÃO DA VAGA aqui", lines=10, placeholder="Cole a descrição completa da vaga...")

    # Lógica para mostrar/ocultar campos baseada no seletor
    def toggle_input_fields(selection):
        if selection == "Arquivo":
            return {
                group_curriculo_arquivo: gr.update(visible=True),
                group_curriculo_texto: gr.update(visible=False),
                group_vaga_arquivo: gr.update(visible=True),
                group_vaga_texto: gr.update(visible=False),
            }
        elif selection == "Texto":
            return {
                group_curriculo_arquivo: gr.update(visible=False),
                group_curriculo_texto: gr.update(visible=True),
                group_vaga_arquivo: gr.update(visible=False), # Vaga pode ser arquivo mesmo se currículo for texto
                group_vaga_texto: gr.update(visible=True),   # Ou texto
            }
        return {} # Default, não deveria acontecer

    input_type_selector.change(
        fn=toggle_input_fields,
        inputs=input_type_selector,
        outputs=[group_curriculo_arquivo, group_curriculo_texto, group_vaga_arquivo, group_vaga_texto]
    )

    gr.Markdown("---") # Separador visual
    btn_analisar = gr.Button("🔍 Analisar Currículo", variant="primary", size="lg")

    output_analise_completa = gr.Markdown(elem_id="output_analise_markdown") # Usar gr.Markdown para renderizar a saída
    # Se quiser um box com scroll:
    # output_analise_completa = gr.Textbox(label="💡 Análise e Sugestões Completas:", lines=20, interactive=False, elem_id="output_analise_textbox", show_copy_button=True)
    # A IA precisa retornar Markdown para o gr.Markdown funcionar bem. Para Textbox, ela pode retornar texto plano.

    btn_analisar.click(
        fn=processar_analise_completa_ui,
        inputs=[
            input_type_selector, # Passa a seleção do Radio
            curriculo_arquivo_input, curriculo_texto_input,
            vaga_arquivo_input, vaga_texto_input
        ],
        outputs=output_analise_completa
    )

    gr.Markdown(
        """
        ---
        <p style="text-align:center; font-size:0.9em; color:grey;">
        Projeto da Imersão IA Alura - Auxiliar de Currículo
        </p>
        """
    )

# Lançamento da interface
print("Iniciando a interface Gradio...")
demo.launch(debug=True, share=True)

Iniciando a interface Gradio...
Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://83fa2e7923ff954244.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


--- Iniciando processar_analise_completa_ui com tipo_input: Arquivo ---
DEBUG UI: Currículo por arquivo: /tmp/gradio/362a0a0dd2659350bb6f88b599406231a69017f0b0018395144007d6283c01c3/Curriculo João Batista.pdf, tipo: pdf
DEBUG UI: Vaga por arquivo: /tmp/gradio/73ff4dd4ffb4cac74286719c6ec87e6683e883c4f4caa79efbe8b2b9f347113d/EngenheiroSmartFit.pdf, tipo: pdf
DEBUG UI: Chamando backend 'analisar_curriculo_completo_para_vaga'
DEBUG - analisar_curriculo_completo_para_vaga: Iniciando com currículo (tipo: pdf) e vaga (tipo: pdf).
DEBUG - obter_texto_entrada: recebendo entrada_raw (tipo: <class 'str'>), tipo_entrada_str: 'pdf'
DEBUG - ler_pdf: Tentando ler '/tmp/gradio/362a0a0dd2659350bb6f88b599406231a69017f0b0018395144007d6283c01c3/Curriculo João Batista.pdf'
DEBUG - ler_pdf: Texto extraído de '/tmp/gradio/362a0a0dd2659350bb6f88b599406231a69017f0b0018395144007d6283c01c3/Curriculo João Batista.pdf' (primeiros 50 chars): 'JOÃO BATISTA  DESENVOLVEDOR BACK-END CARTA DE APRE...'
DEBUG - obter_text

In [ ]:
import os
from google.colab import userdata
import google.generativeai as genai

# Certifique-se que isso é executado ANTES de demo.launch()
try:
    GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
    os.environ['GOOGLE_API_KEY'] = GOOGLE_API_KEY # Para uso com os.environ se necessário
    genai.configure(api_key=GOOGLE_API_KEY)
    print("Chave da API configurada com sucesso!")
except Exception as e:
    print(f"Erro ao configurar a chave da API: {e}")
    # Considere não iniciar a UI se a chave não puder ser configurada

Chave da API configurada com sucesso!


In [ ]:
import gradio as gr
import os


# --- INÍCIO DAS FUNÇÕES DE BACKEND ---
import PyPDF2
import pytesseract
from PIL import Image

import PyPDF2
from PIL import Image
import pytesseract # Certifique-se que pytesseract e o executável Tesseract OCR estão instalados e configurados

# ... (outros imports como os, google.generativeai as genai, etc.)

def ler_pdf(caminho_pdf):
    print(f"DEBUG - ler_pdf: Tentando ler '{caminho_pdf}'")
    try:
        texto_completo = ""
        with open(caminho_pdf, 'rb') as arquivo_pdf:
            leitor_pdf = PyPDF2.PdfReader(arquivo_pdf)
            if not leitor_pdf.pages: # Verifica se o PDF tem páginas
                 print(f"DEBUG - ler_pdf: PDF '{caminho_pdf}' não tem páginas ou está vazio.")
                 return "ErroInterno_PDFVazioOuSemPaginas" # Retorno específico para depuração
            for pagina_num in range(len(leitor_pdf.pages)):
                pagina = leitor_pdf.pages[pagina_num]
                texto_pagina = pagina.extract_text()
                if texto_pagina: # Adiciona texto apenas se extraído
                    texto_completo += texto_pagina + "\n"
        if not texto_completo.strip(): # Se após ler todas as páginas, não há texto útil
            print(f"DEBUG - ler_pdf: Nenhum texto extraído do PDF '{caminho_pdf}'. Pode ser um PDF de imagem sem OCR.")
        texto_preview = texto_completo[:50].replace('\n', ' ')
        print(f"DEBUG - ler_pdf: Texto extraído de '{caminho_pdf}' (primeiros 50 chars): '{texto_preview}...'")
        return texto_completo
    except FileNotFoundError:
        print(f"DEBUG - ler_pdf: Arquivo não encontrado em '{caminho_pdf}'.")
        return "ErroInterno_ArquivoNaoEncontrado"
    except PyPDF2.errors.PdfReadError as e: # Erro específico do PyPDF2
        print(f"DEBUG - ler_pdf: Erro ao ler o PDF '{caminho_pdf}': {e}")
        return f"ErroInterno_LeituraPDF: {str(e)}"
    except Exception as e:
        print(f"DEBUG - ler_pdf: Erro inesperado ao ler o PDF '{caminho_pdf}': {e}")
        return f"ErroInterno_InesperadoPDF: {str(e)}"

def ler_imagem(caminho_imagem):
    print(f"DEBUG - ler_imagem: Tentando ler '{caminho_imagem}' com Tesseract.")
    try:
        # Certifique-se de que o caminho para o executável do Tesseract está no PATH
        # ou configure-o: pytesseract.pytesseract.tesseract_cmd = r'/usr/bin/tesseract' (exemplo Linux)
        texto_extraido = pytesseract.image_to_string(Image.open(caminho_imagem), lang='por') # 'por' para português
        if not texto_extraido.strip():
            print(f"DEBUG - ler_imagem: Nenhum texto extraído da imagem '{caminho_imagem}'.")
            return "ErroInterno_ImagemSemTextoUtil"
        texto_preview = texto_extraido[:50].replace('\n', ' ')
        print(f"DEBUG - ler_imagem: Texto extraído de '{caminho_imagem}' (primeiros 50 chars): '{texto_preview}...'")
        return texto_extraido
    except FileNotFoundError:
        print(f"DEBUG - ler_imagem: Arquivo não encontrado em '{caminho_imagem}'.")
        return "ErroInterno_ArquivoNaoEncontrado"
    except pytesseract.TesseractNotFoundError:
        print("DEBUG - ler_imagem: Erro Crítico - Tesseract OCR não encontrado. Verifique a instalação.")
        return "ErroInterno_TesseractNaoEncontrado"
    except Exception as e:
        print(f"DEBUG - ler_imagem: Erro inesperado ao ler a imagem '{caminho_imagem}': {e}")
        return f"ErroInterno_InesperadoImagem: {str(e)}"

def obter_texto_entrada(entrada_raw, tipo_entrada_str):
    print(f"DEBUG - obter_texto_entrada: recebendo entrada_raw (é arquivo? {'caminho' if isinstance(entrada_raw, str) and (tipo_entrada_str == 'pdf' or tipo_entrada_str == 'imagem') else 'texto'}), tipo_entrada_str: '{tipo_entrada_str}'")
    if tipo_entrada_str == "texto":
        if not isinstance(entrada_raw, str): # Sanity check
             print(f"DEBUG - obter_texto_entrada: Esperava string para tipo 'texto', mas recebeu {type(entrada_raw)}")
             return "ErroInterno_TipoIncorretoParaTexto"
        return entrada_raw
    elif tipo_entrada_str == "pdf":
        return ler_pdf(entrada_raw) # entrada_raw aqui é o caminho do arquivo
    elif tipo_entrada_str == "imagem":
        return ler_imagem(entrada_raw) # entrada_raw aqui é o caminho do arquivo
    else:
        msg_erro = f"ErroInterno_TipoEntradaDesconhecido: '{tipo_entrada_str}'"
        print(f"DEBUG - obter_texto_entrada: {msg_erro}")
        return msg_erro

def adequar_curriculo_vaga(curriculo_input_raw, vaga_input_raw, tipo_curriculo_str, tipo_vaga_str):
    print(f"DEBUG - adequar_curriculo_vaga: Iniciando com currículo (tipo: {tipo_curriculo_str}) e vaga (tipo: {tipo_vaga_str}).")

    # 1. Obter o texto do currículo
    texto_do_curriculo = obter_texto_entrada(curriculo_input_raw, tipo_curriculo_str)
    if isinstance(texto_do_curriculo, str) and texto_do_curriculo.startswith("ErroInterno_"):
        print(f"DEBUG - adequar_curriculo_vaga: Erro ao obter texto do currículo: {texto_do_curriculo}")
        # Mapear erros internos para mensagens mais amigáveis se desejar, ou retornar o erro interno
        return f"Não foi possível processar o arquivo do currículo. (Detalhe: {texto_do_curriculo})"

    # 2. Obter o texto da descrição da vaga
    texto_da_vaga = ""
    if vaga_input_raw and (isinstance(vaga_input_raw, str) and vaga_input_raw.strip()): # Se a vaga foi fornecida (texto ou caminho)
        texto_da_vaga = obter_texto_entrada(vaga_input_raw, tipo_vaga_str)
        if isinstance(texto_da_vaga, str) and texto_da_vaga.startswith("ErroInterno_"):
            print(f"DEBUG - adequar_curriculo_vaga: Erro ao obter texto da vaga: {texto_da_vaga}")
            return f"Não foi possível processar o arquivo da descrição da vaga. (Detalhe: {texto_da_vaga})"
    else:
        print("DEBUG - adequar_curriculo_vaga: Nenhuma descrição de vaga fornecida ou entrada vazia para vaga.")
        texto_da_vaga = "" # Garante que é uma string vazia se não processada

    # --- O restante da função continua igual ---
    instrucao_de_sistema = """
    Você é um especialista em recrutamento e seleção com 15 anos de experiência...
    """ # Seu system instruction completo

    partes_do_prompt = [
        "Analise o seguinte currículo:",
        "--- INÍCIO DO CURRÍCULO ---",
        texto_do_curriculo if texto_do_curriculo else "Currículo não pôde ser lido ou não foi fornecido.",
        "--- FIM DO CURRÍCULO ---"
    ]

    if texto_da_vaga and texto_da_vaga.strip():
        partes_do_prompt.extend([
            "\nCompare-o com a seguinte descrição de vaga:",
            "--- INÍCIO DA DESCRIÇÃO DA VAGA ---",
            texto_da_vaga,
            "--- FIM DA DESCRIÇÃO DA VAGA ---"
        ])
    else:
        partes_do_prompt.append("\nA descrição da vaga não foi fornecida ou não pôde ser lida. Forneça sugestões gerais de melhoria para o currículo.")

    partes_do_prompt.append("\nCom base nisso, forneça suas sugestões de melhoria:")
    prompt_final_para_ia = "\n".join(partes_do_prompt)

    print(f"DEBUG - adequar_curriculo_vaga: Prompt final (primeiros 200 chars): '{prompt_final_para_ia[:200].replace('\n', ' ')}...'")

    sugestoes_da_ia = enviar_prompt_para_ia(prompt_final_para_ia, instrucao_de_sistema)

    if isinstance(sugestoes_da_ia, str) and sugestoes_da_ia.startswith("ErroAPI_"): # Supondo que enviar_prompt_para_ia também use um prefixo de erro
        print(f"DEBUG - adequar_curriculo_vaga: Erro da IA: {sugestoes_da_ia}")
        return f"Ocorreu um erro ao comunicar com a IA: {sugestoes_da_ia}"

    print(f"DEBUG - adequar_curriculo_vaga: Sugestões recebidas da IA (primeiros 100 chars): '{str(sugestoes_da_ia)[:100].replace('\n', ' ')}...'")
    return sugestoes_da_ia

def ajustar_tom_vocabulario(curriculo, descricao_vaga, tipo_entrada_curriculo="texto", tipo_entrada_vaga="texto"):
    """
    Analisa o currículo e a descrição da vaga e ajusta o tom e o vocabulário do currículo.

    Args:
        curriculo (str): O texto do currículo ou o caminho para o arquivo.
        descricao_vaga (str): O texto da descrição da vaga ou o caminho para o arquivo.
        tipo_entrada_curriculo (str): "texto", "pdf" ou "imagem".
        tipo_entrada_vaga (str): "texto", "pdf" ou "imagem".

    Returns:
        str: Sugestões de ajustes no tom e no vocabulário do currículo.
    """
    texto_curriculo = obter_texto_entrada(curriculo, tipo_entrada_curriculo)
    texto_vaga = obter_texto_entrada(descricao_vaga, tipo_entrada_vaga)

    if "Erro" in texto_curriculo:
        return texto_curriculo
    if "Erro" in texto_vaga:
        return texto_vaga

    system_instruction = """
    Você é um especialista em linguagem profissional com foco em recrutamento e seleção.
    Analise o currículo e a descrição da vaga e determine se o tom e o vocabulário do currículo
    são adequados para a vaga. Se não forem, sugira ajustes.
    """
    prompt = f"""
    Currículo:

    {texto_curriculo}

    Descrição da Vaga:
    {texto_vaga}

    O tom da vaga é formal, informal, técnico ou outro?
    O tom do currículo é formal, informal, técnico ou outro
    """
# --- FIM DAS SUAS FUNÇÕES DE BACKEND ---


# Função de processamento para a aba "Adequação à Vaga"
def processar_adequacao_vaga_ui(curriculo_file_obj, curriculo_texto_input, vaga_file_obj, vaga_texto_input):
    print("--- Iniciando processar_adequacao_vaga_ui ---") # DEBUG
    curriculo_final = ""
    tipo_curriculo = ""
    descricao_vaga_final = ""
    tipo_vaga = ""
    erro_msg = []

    # Processar Currículo
    print(f"DEBUG: curriculo_file_obj: {curriculo_file_obj}") # DEBUG
    print(f"DEBUG: curriculo_texto_input: '{curriculo_texto_input}'") # DEBUG
    if curriculo_file_obj is not None:
        curriculo_final = curriculo_file_obj.name
        print(f"DEBUG: Currículo por arquivo: {curriculo_final}") # DEBUG
        ext = os.path.splitext(curriculo_final)[1].lower()
        if ext == ".pdf":
            tipo_curriculo = "pdf"
        elif ext in [".png", ".jpg", ".jpeg", ".webp"]:
            tipo_curriculo = "imagem"
        else:
            erro_msg.append("Erro no Currículo: Tipo de arquivo não suportado.")
    elif curriculo_texto_input and curriculo_texto_input.strip():
        curriculo_final = curriculo_texto_input
        tipo_curriculo = "texto"
        print(f"DEBUG: Currículo por texto.") # DEBUG
    else:
        erro_msg.append("Erro: Forneça o currículo (arquivo ou texto).")

    # Processar Descrição da Vaga
    print(f"DEBUG: vaga_file_obj: {vaga_file_obj}") # DEBUG
    print(f"DEBUG: vaga_texto_input: '{vaga_texto_input}'") # DEBUG
    if vaga_file_obj is not None:
        descricao_vaga_final = vaga_file_obj.name
        print(f"DEBUG: Vaga por arquivo: {descricao_vaga_final}") # DEBUG
        ext = os.path.splitext(descricao_vaga_final)[1].lower()
        if ext == ".pdf":
            tipo_vaga = "pdf"
        elif ext in [".png", ".jpg", ".jpeg", ".webp"]:
            tipo_vaga = "imagem"
        else:
            erro_msg.append("Erro na Vaga: Tipo de arquivo não suportado.")
    elif vaga_texto_input and vaga_texto_input.strip():
        descricao_vaga_final = vaga_texto_input
        tipo_vaga = "texto"
        print(f"DEBUG: Vaga por texto.") # DEBUG
    else:
        descricao_vaga_final = "" # Vaga opcional, string vazia
        tipo_vaga = "texto" # Define como texto para a função backend
        print(f"DEBUG: Vaga não fornecida, tratando como texto vazio.") # DEBUG

    if erro_msg:
        print(f"DEBUG: Erros de validação: {erro_msg}") # DEBUG
        return "\n".join(erro_msg)

    print(f"DEBUG: Chamando adequar_curriculo_vaga com:") # DEBUG
    print(f"  curriculo_final: '{curriculo_final[:100]}...' (tipo: {type(curriculo_final)})") # DEBUG
    print(f"  descricao_vaga_final: '{descricao_vaga_final[:100]}...' (tipo: {type(descricao_vaga_final)})") # DEBUG
    print(f"  tipo_curriculo: {tipo_curriculo}") # DEBUG
    print(f"  tipo_vaga: {tipo_vaga}") # DEBUG

    try:
        resultado = adequar_curriculo_vaga(curriculo_final, descricao_vaga_final, tipo_curriculo, tipo_vaga)
        print(f"DEBUG: Resultado de adequar_curriculo_vaga: '{resultado[:200]}...'") # DEBUG
        return resultado
    except Exception as e:
        error_message = f"Ocorreu um erro no processamento: {str(e)}"
        print(f"DEBUG: Exceção capturada: {error_message}") # DEBUG
        import traceback # Para mais detalhes do erro
        traceback.print_exc() # Imprime o stack trace completo no console
        return error_message

# Função de placeholder para outras funcionalidades
def placeholder_function(texto_entrada):
    return f"Funcionalidade para '{texto_entrada}' em desenvolvimento..."

# Definindo o tema com a cor primária laranja-amarelado
# Gradio usa matizes (hues). "Orange" é um bom ponto de partida.
# Você pode ajustar 'primary_hue' e 'secondary_hue'.
# Outras opções: "amber", "yellow".
theme = gr.themes.Soft(
    primary_hue="orange",
    secondary_hue="yellow",
    neutral_hue="slate",
    font=[gr.themes.GoogleFont("Inter"), "ui-sans-serif", "system-ui", "sans-serif"],
).set(
    # Você pode adicionar CSS customizado aqui se precisar de ajustes finos
    # button_primary_background_fill="linear-gradient(to bottom right, orange, yellow)" # Exemplo mais avançado
)


with gr.Blocks(theme=theme, title="Auxiliar de Currículo IA") as demo:
    gr.Markdown(
        """
        <div style="text-align: center;">
            <h1 style="color: #E67E22;">🚀 Auxiliar de Currículo Inteligente 🚀</h1>
            <p>Potencialize seu currículo com Inteligência Artificial!</p>
        </div>
        """
    )

    with gr.Tabs() as tabs:
        with gr.TabItem("🎯 Adequação à Vaga", id=0):
            gr.Markdown("### Analise seu currículo em relação a uma vaga específica.")
            with gr.Row():
                with gr.Column(scale=1):
                    gr.Markdown("**Seu Currículo**")
                    curriculo_arquivo_input = gr.File(label="Upload Currículo (PDF/Imagem)", file_types=[".pdf", ".png", ".jpg", ".jpeg", ".webp"])
                    curriculo_texto_input = gr.Textbox(label="Ou cole o texto do currículo aqui", lines=10, placeholder="Cole o conteúdo do seu currículo...")
                with gr.Column(scale=1):
                    gr.Markdown("**Descrição da Vaga (Opcional)**")
                    vaga_arquivo_input = gr.File(label="Upload Descrição da Vaga (PDF/Imagem)", file_types=[".pdf", ".png", ".jpg", ".jpeg", ".webp"])
                    vaga_texto_input = gr.Textbox(label="Ou cole o texto da vaga aqui", lines=10, placeholder="Cole a descrição da vaga...")

            btn_adequacao = gr.Button("Analisar Adequação", variant="primary")
            output_adequacao = gr.Textbox(label="💡 Sugestões e Análise:", lines=15, interactive=False)

            btn_adequacao.click(
                fn=processar_adequacao_vaga_ui,
                inputs=[curriculo_arquivo_input, curriculo_texto_input, vaga_arquivo_input, vaga_texto_input],
                outputs=output_adequacao
            )

        with gr.TabItem("🗣️ Tom e Vocabulário", id=1):
            gr.Markdown("### Ajuste o tom e vocabulário do seu currículo.")
            # Adicionar inputs e outputs para esta funcionalidade
            # Exemplo:
            # curriculo_input_tom = gr.Textbox(label="Texto do Currículo", lines=10)
            # vaga_input_tom = gr.Textbox(label="Texto da Vaga (para referência de tom)", lines=5)
            # btn_tom = gr.Button("Analisar Tom/Vocabulário")
            # output_tom = gr.Textbox(label="Sugestões de Tom e Vocabulário")
            # btn_tom.click(fn=sua_funcao_de_tom, inputs=[curriculo_input_tom, vaga_input_tom], outputs=output_tom)
            gr.Markdown("Funcionalidade em desenvolvimento.")


        with gr.TabItem("🔑 Palavras-Chave", id=2):
            gr.Markdown("### Encontre e sugira palavras-chave relevantes.")
            gr.Markdown("Funcionalidade em desenvolvimento.")

        with gr.TabItem("📄 Título e Resumo", id=3):
            gr.Markdown("### Crie um título e resumo profissional impactante.")
            gr.Markdown("Funcionalidade em desenvolvimento.")

        with gr.TabItem("🛠️ Skills e Experiências", id=4):
            gr.Markdown("### Destaque suas principais habilidades e experiências.")
            gr.Markdown("Funcionalidade em desenvolvimento.")

    gr.Markdown(
        """
        ---
        <p style="text-align:center; font-size:0.9em; color:grey;">
        Projeto da Imersão IA Alura - Auxiliar de Currículo
        </p>
        """
    )


# Para rodar no Colab e obter um link público (útil para testar em mobile)
demo.launch(debug=True, share=True)
# Se share=True der problema de conexão/firewall, tente apenas demo.launch(debug=True)
# debug=True ajuda a ver erros no console do Colab

SyntaxError: f-string expression part cannot include a backslash (<ipython-input-37-f1379f55ae24>, line 33)

In [ ]:
from google.colab import files
import os # Adicionado para verificar a extensão do arquivo

def apresentar_funcionalidades():
    """Apresenta as funcionalidades ao usuário e coleta as informações."""
    print("Auxiliar de Currículo Inteligente")
    print("---------------------------------")
    print("Selecione a funcionalidade desejada:")
    print("1. Adequação do Currículo à Vaga: Compara o currículo e a descrição da vaga e sugere ajustes para aumentar a relevância.")
    print("2. Ajuste de Tom/Vocabulário: Ajusta a linguagem do currículo para o tom adequado à vaga.")
    print("3. Sugestão de Palavras-Chave: Identifica e sugere a inclusão de termos importantes da descrição da vaga no currículo.")
    print("4. Sugestão de Título/Resumo Profissional: Gera um resumo atraente e direcionado para a vaga específica.")
    print("5. Análise de Skills e Experiências: Sugere como destacar habilidades e experiências relevantes para a vaga.")
    print("---------------------------------")

    opcao_funcionalidade = input("Digite o número da funcionalidade desejada: ")

    curriculo_input = ""
    descricao_vaga_input = ""
    tipo_entrada_curriculo = ""
    tipo_entrada_vaga = ""

    if opcao_funcionalidade == "1": # Foco na funcionalidade de Adequação
        print("\n--- Adequação do Currículo à Vaga ---")
        print("Você pode fornecer o currículo e a descrição da vaga como texto ou arquivo (PDF/Imagem).")

        # Coletando informações do Currículo
        print("\n[Currículo]")
        opcao_curriculo = input("Como deseja fornecer o currículo? (1 para texto, 2 para arquivo): ")
        if opcao_curriculo == "1":
            curriculo_input = input("Digite o texto do seu currículo:\n")
            tipo_entrada_curriculo = "texto"
        elif opcao_curriculo == "2":
            print("Faça o upload do arquivo do seu currículo (PDF ou Imagem):")
            uploaded_curriculo = files.upload()
            if uploaded_curriculo:
                curriculo_input = list(uploaded_curriculo.keys())[0] # Pega o nome do primeiro arquivo enviado
                extensao_curriculo = os.path.splitext(curriculo_input)[1].lower()
                if extensao_curriculo == ".pdf":
                    tipo_entrada_curriculo = "pdf"
                elif extensao_curriculo in [".png", ".jpg", ".jpeg"]:
                    tipo_entrada_curriculo = "imagem"
                else:
                    print("Formato de arquivo do currículo não suportado. Use PDF ou Imagem (PNG, JPG, JPEG).")
                    return
                print(f"Arquivo do currículo '{curriculo_input}' carregado.")
            else:
                print("Nenhum arquivo de currículo foi enviado.")
                return
        else:
            print("Opção inválida para o currículo.")
            return

        # Coletando informações da Descrição da Vaga
        print("\n[Descrição da Vaga]")
        opcao_vaga = input("Como deseja fornecer a descrição da vaga? (1 para texto, 2 para arquivo, ou deixe em branco se não aplicável): ")
        if opcao_vaga == "1":
            descricao_vaga_input = input("Digite o texto da descrição da vaga:\n")
            tipo_entrada_vaga = "texto"
        elif opcao_vaga == "2":
            print("Faça o upload do arquivo da descrição da vaga (PDF ou Imagem):")
            uploaded_vaga = files.upload()
            if uploaded_vaga:
                descricao_vaga_input = list(uploaded_vaga.keys())[0] # Pega o nome do primeiro arquivo enviado
                extensao_vaga = os.path.splitext(descricao_vaga_input)[1].lower()
                if extensao_vaga == ".pdf":
                    tipo_entrada_vaga = "pdf"
                elif extensao_vaga in [".png", ".jpg", ".jpeg"]:
                    tipo_entrada_vaga = "imagem"
                else:
                    print("Formato de arquivo da vaga não suportado. Use PDF ou Imagem (PNG, JPG, JPEG).")
                    return
                print(f"Arquivo da vaga '{descricao_vaga_input}' carregado.")
            else:
                print("Nenhum arquivo de descrição da vaga foi enviado.")
                # Se não enviar arquivo de vaga, tratamos como texto vazio para não dar erro
                descricao_vaga_input = ""
                tipo_entrada_vaga = "texto"
        elif opcao_vaga == "": # Usuário deixou em branco
            print("Nenhuma descrição de vaga fornecida.")
            descricao_vaga_input = ""
            tipo_entrada_vaga = "texto" # Importante definir para não dar erro
        else:
            print("Opção inválida para a descrição da vaga.")
            return

        # Executando a funcionalidade de Adequação
        print("\nAnalisando e gerando sugestões...")
        resultado = adequar_curriculo_vaga(curriculo_input, descricao_vaga_input, tipo_entrada_curriculo, tipo_entrada_vaga)
        print("\n--- Sugestões de Adequação do Currículo ---")
        print(resultado)
        print("-------------------------------------------")

    elif opcao_funcionalidade in ["2", "3", "4", "5"]:
        # Implementar a lógica de coleta de dados para as outras funcionalidades de forma similar
        # Por agora, vamos manter o foco na funcionalidade 1
        print(f"\nFuncionalidade {opcao_funcionalidade} selecionada.")
        print("A lógica de entrada para esta funcionalidade ainda precisa ser detalhada como a opção 1.")
        # Exemplo simplificado para as outras (precisa adaptar):
        # curriculo_input = input("Digite o texto do currículo ou caminho do arquivo: ")
        # tipo_entrada_curriculo = input("Tipo de entrada do currículo (texto/pdf/imagem): ").lower()
        # if opcao_funcionalidade == "2":
        #     descricao_vaga_input = input("Digite o texto da vaga ou caminho do arquivo (opcional): ")
        #     tipo_entrada_vaga = input("Tipo de entrada da vaga (texto/pdf/imagem, ou 'texto' se vazio): ").lower() if descricao_vaga_input else "texto"
        #     resultado = ajustar_tom_vocabulario(curriculo_input, descricao_vaga_input, tipo_entrada_curriculo, tipo_entrada_vaga)
        #     print("\nSugestões de Tom/Vocabulário:")
        #     print(resultado)
        # (Adicionar lógica para as outras funcionalidades aqui)

    else:
        print("Opção de funcionalidade inválida.")

# Lembre-se de ter as funções de leitura de PDF/Imagem e a `enviar_prompt_para_ia` definidas anteriormente.
# Exemplo:
# def ler_pdf(caminho_pdf): ...
# def ler_imagem(caminho_imagem): ...
# def obter_texto_entrada(entrada, tipo_entrada): ...
# def enviar_prompt_para_ia(prompt, system_instruction): ...
# def adequar_curriculo_vaga(curriculo, descricao_vaga, tipo_entrada_curriculo, tipo_entrada_vaga): ...
# (e as outras funções de análise)

# Chamando a função para iniciar o processo
apresentar_funcionalidades()

Auxiliar de Currículo Inteligente
---------------------------------
Selecione a funcionalidade desejada:
1. Adequação do Currículo à Vaga: Compara o currículo e a descrição da vaga e sugere ajustes para aumentar a relevância.
2. Ajuste de Tom/Vocabulário: Ajusta a linguagem do currículo para o tom adequado à vaga.
3. Sugestão de Palavras-Chave: Identifica e sugere a inclusão de termos importantes da descrição da vaga no currículo.
4. Sugestão de Título/Resumo Profissional: Gera um resumo atraente e direcionado para a vaga específica.
5. Análise de Skills e Experiências: Sugere como destacar habilidades e experiências relevantes para a vaga.
---------------------------------
Digite o número da funcionalidade desejada: 1

--- Adequação do Currículo à Vaga ---
Você pode fornecer o currículo e a descrição da vaga como texto ou arquivo (PDF/Imagem).

[Currículo]
Como deseja fornecer o currículo? (1 para texto, 2 para arquivo): 2
Faça o upload do arquivo do seu currículo (PDF ou Imagem):


Saving Curriculo João Batista.pdf to Curriculo João Batista (2).pdf
Arquivo do currículo 'Curriculo João Batista (2).pdf' carregado.

[Descrição da Vaga]
Como deseja fornecer a descrição da vaga? (1 para texto, 2 para arquivo, ou deixe em branco se não aplicável): 1
Digite o texto da descrição da vaga:
Executivo de Negócios Pleno, Presencial   Como é o ambiente de trabalho? Se seu objetivo principal é ganhar dinheiro rápido. Se você topa fazer algo que não goste pela recompensa financeira. Se você quer ser apenas mais um dentro da empresa. ESSA VAGA NÃO É PARA VOCÊ! Queremos pessoas excelentes, com sangue nos olhos e espírito de dono. Desejamos mais um sócio empreendedor e não apenas um colaborador! Se você não se identifica com os valores abaixo, não precisa nem continuar o processo seletivo, temos certeza que você será mais feliz em outro lugar :)    Como uma empresa nova e com espírito empreendedor precisamos de pessoas que pensem fora da caixa e sejam capazes de entregar sempre mai

In [ ]:
def enviar_prompt_para_ia(prompt, system_instruction="", model_name="gemini-2.0-flash"):
    """
    Envia um prompt para a API do Google AI e retorna a resposta.

    Args:
        prompt (str): O prompt do usuário.
        system_instruction (str): Instruções para o modelo (opcional).
        model_name (str): O nome do modelo a ser usado (padrão: "gemini-2.0-flash").

    Returns:
        str: A resposta da API.
    """
    try:
        # Obtenha a chave da API da variável de ambiente
        api_key = os.environ.get('GOOGLE_API_KEY')
        if not api_key:
            return "Erro: Chave da API não encontrada. Certifique-se de defini-la nos Secrets do Colab."

        client = genai.Client(api_key=api_key)

        # Configuração do chat com instruções de sistema (se fornecidas)
        if system_instruction:
            chat_config = types.GenerateContentConfig(system_instruction=system_instruction)
            chat = client.chats.create(model=model_name, config=chat_config)
        else:
            chat = client.chats.create(model=model_name)

        # Envia o prompt e recebe a resposta
        response = chat.send_message(prompt)
        return response.text

    except Exception as e:
        return f"Erro ao enviar o prompt: {e}"

In [ ]:
prompt_teste_2 = "O que é um currículo?"
system_instruction = "Você é um especialista em recrutamento e seleção."
resposta = enviar_prompt_para_ia(prompt_teste_2, system_instruction=system_instruction)
print(resposta)

A capital do Brasil é **Brasília**.

Como especialista em recrutamento e seleção, posso te dizer que o currículo é uma ferramenta fundamental na busca por um emprego. Ele é um documento que resume sua trajetória profissional, suas habilidades, sua formação acadêmica e outras informações relevantes para a vaga que você está buscando.

Pense no currículo como o seu cartão de visitas profissional. Ele é a primeira impressão que você causa em um recrutador, e é através dele que você tem a chance de mostrar por que é o candidato ideal para a posição.

**Em resumo, o currículo serve para:**

*   **Apresentar você:** Fornece uma visão geral da sua experiência e qualificações.
*   **Demonstrar sua adequação à vaga:** Destaca as habilidades e experiências mais relevantes para a posição desejada.
*   **Convencer o recrutador a te chamar para uma entrevista:** Desperta o interesse do recrutador em te conhecer melhor.

**O que geralmente inclui um currículo:**

*   **Dados pessoais:** Nome complet